In [1]:
import pandas as pd, ipywidgets as widgets, numpy as np, matplotlib.pyplot as plt, io
raw_uploader, eln_uploader = widgets.FileUpload(multiple = False), widgets.FileUpload(multiple = False)
first_population_widget = widgets.Text(placeholder='First Pop',description='First Population:', disabled=False)
second_population_widget = widgets.Text(placeholder='Second Pop',description='Second Population:', disabled=False)
display("Upload iQue3 Data (csv)")
display(raw_uploader)
display("Upload ELN w/ Antibodies Sheet (xlsx)")
display(eln_uploader)
display(first_population_widget)
display(second_population_widget)

'Upload iQue3 Data (csv)'

FileUpload(value={}, description='Upload')

'Upload ELN w/ Antibodies Sheet (xlsx)'

FileUpload(value={}, description='Upload')

Text(value='', description='First Population:', placeholder='First Pop')

Text(value='', description='Second Population:', placeholder='Second Pop')

In [2]:
iquename, elnname = list(raw_uploader.value.keys())[0], list(eln_uploader.value.keys())[0]
iquedata = pd.read_csv(io.BytesIO(raw_uploader.value[iquename]['content']))
eln = pd.read_excel(io.BytesIO(eln_uploader.value[elnname]['content']))
ab_sheet = pd.read_excel(io.BytesIO(eln_uploader.value[elnname]['content']), sheet_name = "Antibody Plate Map")
first_population_name = first_population_widget.value
second_population_name = second_population_widget.value
iquedata.rename(columns = {'Plate' : 'iQue3 Plate','Well ID':'iQue3 384 Well ID'}, inplace = True)
ab_sheet.rename(columns = {'Plate' : 'Antibody Plate','Well ID':'Antibody Platemap 96 Well ID'}, inplace = True)
iquedata

,Experiment,Analysis,iQue3 Plate,Export Date and Time,iQue3 384 Well ID,Row,Column,Well Type,Count of All Wells,Median BL1-H of singlets,Median RL1-H of singlets,Count of singlets
0,22_10_10 A2a Primary Screen,Analysis 1,Plate 1,10/18/22 11:06,A01,A,1,A2a,4383,2138419.0,957998.5,664
1,22_10_10 A2a Primary Screen,Analysis 1,Plate 1,10/18/22 11:06,B01,B,1,A2a,2721,2212419.5,903494.5,378
2,22_10_10 A2a Primary Screen,Analysis 1,Plate 1,10/18/22 11:06,C01,C,1,A2a,2993,1434206.0,800112.0,383
3,22_10_10 A2a Primary Screen,Analysis 1,Plate 1,10/18/22 11:06,D01,D,1,A2a,3516,2862318.5,831600.0,636
4,22_10_10 A2a Primary Screen,Analysis 1,Plate 1,10/18/22 11:06,E01,E,1,A2a,2974,2705913.0,1460251.5,456
...,...,...,...,...,...,...,...,...,...,...,...,...
1147,22_10_10 A2a Primary Screen,Analysis 1,Plate 3,10/18/22 11:06,L24,L,24,Negative,1225,10349.0,6609.0,431
1148,22_10_10 A2a Primary Screen,Analysis 1,Plate 3,10/18/22 11:06,M24,M,24,Negative,1132,10401.0,6573.0,349
1149,22_10_10 A2a Primary Screen,Analysis 1,Plate 3,10/18/22 11:06,N24,N,24,Negative,1109,10248.0,6967.0,406
1150,22_10_10 A2a Primary Screen,Analysis 1,Plate 3,10/18/22 11:06,O24,O,24,Negative,1617,10305.0,7623.0,757


In [3]:
data_cleaned = iquedata[["iQue3 Plate", "iQue3 384 Well ID", "Well Type", "Median RL1-H of singlets", "Count of singlets"]]
data_cleaned

,iQue3 Plate,iQue3 384 Well ID,Well Type,Median RL1-H of singlets,Count of singlets
0,Plate 1,A01,A2a,957998.5,664
1,Plate 1,B01,A2a,903494.5,378
2,Plate 1,C01,A2a,800112.0,383
3,Plate 1,D01,A2a,831600.0,636
4,Plate 1,E01,A2a,1460251.5,456
...,...,...,...,...,...
1147,Plate 3,L24,Negative,6609.0,431
1148,Plate 3,M24,Negative,6573.0,349
1149,Plate 3,N24,Negative,6967.0,406
1150,Plate 3,O24,Negative,7623.0,757


In [4]:
#Return data frame with only samples of the well type specified
def clean_for_well_type(well_type_string):
    return data_cleaned[data_cleaned["Well Type"] == well_type_string]

#Average function that takes a string indicating the well type of interest (marked in iQue3 software, or manually changed
#Separates n=1 and n=2 replicates from 384 well format, resets indeces, and averages
#Returns a series with the averaged MFI ratios, removing the replicate wells
def average_MFI(well_type_string):
    selected_wells = clean_for_well_type(well_type_string)
    MFIs = selected_wells["Median RL1-H of singlets"]
    firsts = MFIs.iloc[::2]
    seconds = MFIs.iloc[1::2]
    firsts.reset_index(inplace = True, drop = True)
    seconds.reset_index(inplace = True, drop = True)
    average = firsts + seconds
    average = average/2
    return average

#Calculates the MFI ratio given a positive and negative series, 
#and appends the MFI ratio to the original data frame, excluding the negative population
#positive_series: the series of the positive values
#negative_series: the series of the negative (untransfected) values
#positive_string: the string that marks the positive samples in the Well Type column from iQue3 csv
def calculate_analysis_sheet_2pop(first_pop_series, second_pop_series, negative_series, first_population_string, second_population_string):
    first_pop_MFI_ratio = first_pop_series/negative_series
    second_pop_MFI_ratio = second_pop_series/negative_series
    original_positive = clean_for_well_type(first_population_string)
    original_positive = original_positive[::2]
    original_positive.reset_index(inplace = True, drop = True)
    original_positive[first_population_string + " MFI Ratio"] = first_pop_MFI_ratio
    original_positive[first_population_string + " Averaged Raw MFI"] = first_pop_series
    original_positive[second_population_string + " MFI Ratio"] = second_pop_MFI_ratio
    original_positive[second_population_string + " Averaged Raw MFI"] = second_pop_series
    original_positive["Parental Averaged Raw MFI"] = negative_series
    analysis_sheet = pd.concat([original_positive, ab_sheet], axis = 1)
    analysis_sheet.index += 1
    analysis_sheet.drop(columns = ['Median RL1-H of singlets'], inplace = True)
    return analysis_sheet
# analysis_sheet[["iQue3 Plate", "iQue3 384 Well ID", "Well Type", first_population_string + " MFI Ratio", 
#                            second_population_string + " MFI Ratio", 
#                            "Antibody", "Concentration"]] 

In [5]:
averageFirstPopulation = average_MFI(first_population_name)
averageSecondPopulation = average_MFI(second_population_name)
averageParental = average_MFI("Negative")

In [6]:
#Create new DF for MFI ratios(don't modify originals)
analyzed_data = calculate_analysis_sheet_2pop(averageFirstPopulation, averageSecondPopulation,
                                        averageParental, first_population_name, second_population_name)
analyzed_data.to_excel("Flow Binding Primary Screen.xlsx", sheet_name='Analyzed Data')
analyzed_data
# AllLeadsSorted = analyzed_data.sort_values(by= first_population_name + " MFI Ratio", ascending = False)
# AllLeadsSorted

,iQue3 Plate,iQue3 384 Well ID,Well Type,Count of singlets,A2a MFI Ratio,A2a Averaged Raw MFI,A2b MFI Ratio,A2b Averaged Raw MFI,Parental Averaged Raw MFI,Lot #,...,Protein Name,E1,Protein Concentration (ug/uL),Volume (uL),Yield (ug),Date Purified,BPP,Experiment Drug Plate,Experiment Well,Antibody Plate
1,Plate 1,A01,A2a,664,32.564648,930746.50,2.463158,70400.75,28581.50,56250.0,...,PRJ133_015,13.7,0.93,240.0,223.2,9.26.22,BPP-618,1.0,A1,BPP-618 Plate 1
2,Plate 1,C01,A2a,383,40.433949,815856.00,1.912613,38591.75,20177.50,56251.0,...,PRJ133_016,13.7,0.10,240.0,24.0,9.26.22,BPP-618,1.0,B1,BPP-618 Plate 1
3,Plate 1,E01,A2a,456,31.343387,1424494.25,5.111798,232321.00,45448.00,56252.0,...,PRJ133_017,13.7,0.24,240.0,57.6,9.26.22,BPP-618,1.0,C1,BPP-618 Plate 1
4,Plate 1,G01,A2a,371,26.231775,1058832.50,3.375280,136241.50,40364.50,56253.0,...,PRJ133_018,13.7,0.86,240.0,206.4,9.26.22,BPP-618,1.0,D1,BPP-618 Plate 1
5,Plate 1,I01,A2a,337,55.922923,845988.00,2.224604,33653.25,15127.75,56254.0,...,PRJ133_019,13.7,0.87,240.0,208.8,9.26.22,BPP-618,1.0,E1,BPP-618 Plate 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,Plate 3,G08,A2a,492,1.180420,7828.25,1.212199,8039.00,6631.75,NaN,...,Blank,NaN,NaN,NaN,NaN,NaN,NaN,3.0,D8,NaN
189,Plate 3,I08,A2a,1416,1.467422,9228.25,1.333492,8386.00,6288.75,NaN,...,Blank,NaN,NaN,NaN,NaN,NaN,NaN,3.0,E8,NaN
190,Plate 3,K08,A2a,1575,4.471573,29238.50,1.167196,7632.00,6538.75,NaN,...,Blank,NaN,NaN,NaN,NaN,NaN,NaN,3.0,F8,NaN
191,Plate 3,M08,A2a,1045,1.279911,8665.00,1.104136,7475.00,6770.00,NaN,...,Blank,NaN,NaN,NaN,NaN,NaN,NaN,3.0,G8,NaN


In [ ]:
# #Sort DF by highest MFI ratio (top hits)
# AllLeadsSorted = MFIRatio_Antibody_cleaned.sort_values(by="MFI Ratio", ascending = False)
# Top10 = MFIRatio_Antibody_cleaned.nlargest(10, "MFI Ratio")
# #Top10 = Top10.append(MFIRatio_Antibody_cleaned.loc[MFIRatio_Antibody_cleaned["DNA"] == "SC-175-017"])
# fig, ax = plt.subplots()
# ax.scatter(x = AllLeadsSorted["Protein Concentration (ug/uL)"], y=AllLeadsSorted["MFI Ratio"], alpha = .2, color = "blue")
# ax.scatter(x = Top10["Protein Concentration (ug/uL)"], y = Top10["MFI Ratio"], alpha = .8, color = "blue", label = "Top Hits")
# ax.set_title("CD40 Primary Screen")
# ax.set_xlabel("Concentration (mg/ml)")
# ax.set_ylabel("MFI Ratio \n Transfected/Untransfected")
# ax.legend()